## Общее описание данных

**Вид на таблицу из "космоса"**

Таблица состоит из двух листов.


*Лист 1: transactions*

Содержит в себе описание транзакций и ID товаров, что говорит о том что по крайней мере таблица не приведена к 3НФ так как
присутствует транзитивная зависимость между ID транзакции и описанием продукта, так как описание продукта относится к ID продукта
а не к ID транзакции, следовательно в основной таблице надо оставить только ID товара, а всю остальную информацию о товарах вынести отдельно

*Лист 2: customers* \n

Таблица не приведена в 1НФ так как есть колонка address которая содержит в себе 2 значения в каждом поле: номер дома и название улицы или района
Все поля колонки address имеют одинаковый паттент <цифры>_пробел_<символы>, что дает возможность разделения через обычный split или регулярные выражения
Далее таблицу нужно будет привести к 3НФ, отделив колонки, касающиеся адреса в отдельную таблицу и оставить только почтовые коды.

План такой:
1) Перевести таблицы со страниц в CSV
2) Загрузить в pandas
3) Разделить колонку address
4) Создать 4 таблицы pandas
5) Создать таблицы в PostgreSQL через библиотеку psycopg2 (взято из материалов модуля)
6) Наполнить таблицы через DBeaver

In [ ]:
"""
    
    Рассмотрим каждую колонку отдельно и сразу сгруппируем их по будущим таблицам:

    customer:

    Колонки сгруппированы для приведения к 3НФ

    Таблица БД customers

    - customer_id ----------->----- Инкремент, первичный ключ, тип данных int, not null
    - first_name ------------>----- Имя, text
    - last_name ------------->----- Фамилия, text
    - gender ---------------->----- Пол, text (встречаются пропуски и нестандартные male/female)
    - DOB ------------------->----- Дата рождения, varchar
    - job_title ------------->----- Должность, text
    - job_industry_category ->----- Индустрия, text
    - wealth_segment -------->----- Благосостояние, text
    - deceased_indicator ---->----- Некий индикатор, varchar
    - owns_car -------------->----- Транспорт, bool
    - postcode -------------->----- Почтовый индекс, int

     Талица БД address
    - postcode -------------->----- Первичный ключ, тип данных int, not null
    - address_house --------->----- Первая часть бывшей колонки addres, int
    - address_street -------->----- Вторая часть бывшей колонки addres, text
    - state ----------------->----- Штат, text
    - country --------------->----- Страна, text
    - property_valuation ---->----- Оценка недвижимости, int


    transactions:
    
    Таблица БД transactions
    
    - transaction_id ------->----- Инкремент, первичный ключ, тип данных int, not null
    - customer_id ---------->----- ID покупателя, тип данных int, nut null
    - transaction_date ----->----- Дата транзации, varchar
    - online_order --------->----- Онлайн заказ, bool, not null 
    - order_status --------->----- Статус заказа, varchar, not null
    
    
    Таблица БД products    
    
    - product_id ----------->----- Первичный ключ, тип данных int, not null
    - brand ---------------->----- Бренд, text
    - product_line --------->----- Ассортиментная группа, text
    - product_class -------->----- Класс продукта, text
    - product_size --------->----- Размер продукта, text
    - list_price ----------->----- Рекомендованая цена, decimal
    - standard_cost -------->----- Стандартная цена, decimal
    
"""

In [101]:
# импортируем зависимости

import psycopg2
import pandas as pd
import numpy as np

In [84]:
# создаем датафреймы для двух CSV из Exel

df_customer_s = pd.read_csv('asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_/customer-Table 1.csv', sep=';')
df_transaction_s = pd.read_csv('asset-v1_SkillFactory+MFTIDS+SEP2023+type@asset+block@customer_and_transaction__2_/transaction-Table 1.csv', sep=';')


In [85]:
# у 1378 продуктов нет ID, оно равно 0, а значит product ID не может быть первичным ключом 
# ТЕМ НЕ МЕНЕЕ я вынесу продукты в отдельную таблицу, потому что, по всей видимости это предполагается как часть задания

df_transaction_s[df_transaction_s['product_id'] == 0]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
34,35,0,2171,8/20/17,False,Approved,Norco Bicycles,Road,medium,medium,"544,05","376,84"
39,40,0,2448,11/28/17,True,Approved,Norco Bicycles,Road,medium,medium,"544,05","376,84"
54,55,0,3140,9/18/17,False,Approved,Norco Bicycles,Standard,low,medium,"363,01","290,41"
60,61,0,1839,2/24/17,False,Approved,OHM Cycles,Road,high,large,"12,01","7,21"
63,64,0,2000,7/8/17,False,Approved,Trek Bicycles,Standard,medium,medium,"499,53","388,72"
...,...,...,...,...,...,...,...,...,...,...,...,...
19921,19922,0,1869,1/6/17,True,Approved,Norco Bicycles,Road,medium,medium,"543,39","407,54"
19941,19942,0,966,7/30/17,True,Approved,OHM Cycles,Standard,low,medium,"71,16","56,93"
19967,19968,0,2751,4/6/17,False,Approved,WeareA2B,Standard,medium,medium,"60,34","45,26"
19987,19988,0,13,4/5/17,True,Approved,Norco Bicycles,Road,medium,medium,"544,05","376,84"


In [86]:
# разделяем колонку address

df_customer_s['address_house'] = df_customer_s['address'].str.extract('(\d+)')
df_customer_s['address_street'] = df_customer_s['address'].str.extract('(\D+)')

# убираем исходную колонку

df_customer_s.drop(columns='address')

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,postcode,state,country,property_valuation,address_house,address_street
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,2016,New South Wales,Australia,10,060,Morning Avenue
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,2153,New South Wales,Australia,10,6,Meadow Vale Court
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,4211,QLD,Australia,9,0,Holy Cross Court
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,2448,New South Wales,Australia,4,17979,Del Mar Point
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,3216,VIC,Australia,9,9,Oakridge Court
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3996,Rosalia,Halgarth,Female,1975-08-09,VP Product Management,Health,Mass Customer,N,No,4511,QLD,Australia,6,57042,Village Green Point
3996,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,2756,NSW,Australia,10,87,Crescent Oaks Alley
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,4032,QLD,Australia,7,8194,Lien Street
3998,3999,Patrizius,NaN,Male,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,2251,NSW,Australia,7,320,Acker Drive


In [87]:
#проверим уникальность каждой записи в первичных ключах customer

df_customer_s['customer_id'].nunique() == len(df_customer_s)

True

In [88]:
#проверим уникальность каждой записи в первичных ключах transaction
df_transaction_s['transaction_id'].nunique() == len(df_transaction_s)

True

In [89]:
# создадим два датафрейма из df_customer_s

products_cols = [
    'product_id',
    'brand',
    'product_line',
    'product_class',
    'product_size',
    'list_price',
    'standard_cost'
    ]

address_cols = [
    'postcode',
    'address_house',
    'address_street',
    'state',
    'country',
    'property_valuation'
    ]

customers_colls = [
    'customer_id',
    'first_name',
    'last_name',
    'gender',
    'DOB',
    'job_title',
    'job_industry_category',
    'wealth_segment',
    'deceased_indicator',
    'owns_car',
    'postcode'
    ]

transaction_colls = [
    'transaction_id',
    'customer_id',
    'product_id',
    'transaction_date',
    'online_order',
    'order_status']

df_product = df_transaction_s[products_cols]
df_address = df_customer_s[address_cols]
df_customers = df_customer_s[customers_colls]
df_transactions = df_transaction_s[transaction_colls]

In [90]:
# чистим дополнительные таблицы от дубликатов

df_product = df_product.drop_duplicates()
df_address = df_address.drop_duplicates()

In [91]:
# продукты сократились до 397 уникальных строк (ИГНОРИРУЕМ ЧТО ТАМ НУЛИ И ЭТО ВООБЩЕ НЕ ГОДИТСЯ ДЛЯ ПЕРВИЧНОГО КЛЮЧА)
len(df_product)

397

In [92]:
#видим что цены указаны через запятую, поэтому они загрузились как str, переводим в другой формат
df_product.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397 entries, 0 to 19871
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     397 non-null    int64 
 1   brand          201 non-null    object
 2   product_line   201 non-null    object
 3   product_class  201 non-null    object
 4   product_size   201 non-null    object
 5   list_price     397 non-null    object
 6   standard_cost  201 non-null    object
dtypes: int64(1), object(6)
memory usage: 24.8+ KB


In [94]:
# Преобразование 'list_price' из строки с запятой в число с плавающей точкой
df_product['list_price'] = df_product['list_price'].apply(lambda x: float(x.replace(',', '.')))

In [102]:
# Преобразование 'list_price' в число с плавающей точкой, обработка NaN
df_product['standard_cost'] = df_product['standard_cost'].apply(lambda x: float(x.replace(',', '.')) if pd.notnull(x) else np.nan)

In [83]:
df_product['list_price']

0          7149
1        209147
2        179343
3        119846
4         17653
          ...  
19340     74454
19383    109818
19793     86856
19859    149743
19871     86792
Name: list_price, Length: 397, dtype: object

In [62]:
# все адреса остались уникальными 
len(df_address)

4000

In [31]:
# посмотрим что с ключом адреса 
df_address['postcode'].nunique()

# упс их всего 874, и это не годится для использования в качестве ключа, так как одному пост-коду может соответствовать разное количество адресов
# следовательно это не однозначное соответствие, следовательно надо или дополнить пост-код номером дома или вообще отказаться от идеи выносить адрес
# в отдельную таблицу

874

In [36]:
# переведем пост-код в строковое значение
df_address['postcode'] = df_address['postcode'].astype(str)

In [37]:
# попробуем создать ключ искусственно по формуле почтовый код + номер дома через '-'
df_address['post_key'] = df_address['postcode'] + '-' + df_address['address_house']

In [50]:
# 64 записи все еще дублируются
len(df_address) - df_address['post_key'].nunique()

64

In [ ]:
#  вывод: нормализация не возможна, так как в адресе нет первичного ключа, оставляем адрес вместе с кастомерами

In [27]:
df_product[df_product['product_id'] == 0]

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost
34,0,Norco Bicycles,Road,medium,medium,"544,05","376,84"
54,0,Norco Bicycles,Standard,low,medium,"363,01","290,41"
60,0,OHM Cycles,Road,high,large,"12,01","7,21"
63,0,Trek Bicycles,Standard,medium,medium,"499,53","388,72"
82,0,OHM Cycles,Standard,medium,medium,"235,63","125,07"
...,...,...,...,...,...,...,...
19340,0,NaN,NaN,NaN,NaN,"744,54",NaN
19383,0,NaN,NaN,NaN,NaN,"1098,18",NaN
19793,0,NaN,NaN,NaN,NaN,"868,56",NaN
19859,0,NaN,NaN,NaN,NaN,"1497,43",NaN


In [51]:
len(df_product) - df_product['product_id'].nunique()

296

In [ ]:
# в списке продуктов так же есть неоднозначность и нет уникального первичного ключа.

### Общий вывод

Приведение к третьей нормальной форме не возможно, технически мы уже в ней, так как у нас нет уникальных ключей из которых можно было бы сделать первичные ключи для всего датасета.

## Решение

В целях выполнения задания я все же оставлю структуру из 4 таблиц, как видно у меня на скринах, но наполнение этих таблиц я буду делать искусственно 
ограниченным количеством записей черех INSERT INTO

In [103]:
# сохраняем полученные таблицы в CSV в ту же директорию что и исходные данные

df_product.to_csv('product.csv', sep=',', index=False)
df_customers.to_csv('customers.csv', sep=',', index=False)
df_transactions.to_csv('transactions.csv', sep=',', index=False)
df_address.to_csv('addres_book.csv', sep=',', index=False)

In [64]:
# теперь создаем подключение к базе

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="MY_defence_2024",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()

In [73]:
# создаем таблицу транзакций

sql = """
CREATE TABLE HW1_transactions (
    transacrion_id SERIAL PRIMARY KEY
    ,customer_id INT
    ,product_id INT
    ,transaction_date VARCHAR
    ,online_order BOOL
    ,order_status VARCHAR
    ); """

cursor.execute(sql)
conn.commit()

In [74]:
# создаем таблицу кастомеров

sql = """
CREATE TABLE HW1_customers (
    customer_id SERIAL PRIMARY KEY
    ,first_name TEXT
    ,last_name TEXT
    ,gender TEXT
    ,DOB VARCHAR
    ,job_title TEXT
    ,job_industry_category TEXT
    ,wealth_segment TEXT
    ,deceased_indicator VARCHAR
    ,owns_car BOOL
    ,postcode INT
    ); """

cursor.execute(sql)
conn.commit()

In [79]:
# создаем таблицу адресов

sql = """
CREATE TABLE HW1_address_book (
    postcode INT NOT NULL PRIMARY KEY
    ,address_house INT
    ,address_street TEXT
    ,state TEXT
    ,country TEXT
    ,property_valuation INT
    ); """

cursor.execute(sql)
conn.commit()

In [76]:
# создаем таблицу товаров

sql = """
CREATE TABLE HW1_products (
    product_id INT NOT NULL PRIMARY KEY
    ,brand TEXT
    ,product_line TEXT
    ,product_class TEXT
    ,product_size TEXT
    ,list_price DECIMAL
    ,standard_cost DECIMAL
    ); """

cursor.execute(sql)
conn.commit()

In [ ]:
# Ура, таблицы созданы, данные готовы для загрузки, хотя бы часть
# В приложенных файлах есть скрины дизайна базы и скрины из DBeaver

In [70]:
conn.rollback()